In [1]:
import pandas as pd #for data cleaning
import numpy as np #for numerical data handling
import matplotlib.pyplot as plt #for visualization
import seaborn as sns #for advanced visualization
from scipy import stats #statistical tests

# **Data Cleaning**

## **Extract data from customer sheet in messy excel**


In [8]:
file_path = 'Bank_Churn_Messy.xlsx'

In [9]:
customer_df = pd.read_excel(file_path,sheet_name=0)
customer_df = pd.DataFrame(customer_df)

In [10]:
customer_df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,EstimatedSalary
0,15634602,Hargrave,619,FRA,Female,42.0,2,€101348.88
1,15647311,Hill,608,Spain,Female,41.0,1,€112542.58
2,15619304,Onio,502,French,Female,42.0,8,€113931.57
3,15701354,Boni,699,FRA,Female,39.0,1,€93826.63
4,15737888,Mitchell,850,Spain,Female,43.0,2,€79084.1


## **CustomerID Type Checking**

In [11]:
customer_df['CustomerId'].dtype

dtype('int64')

In [12]:
customer_df['CustomerId'] = customer_df['CustomerId'].astype('string')

In [13]:
customer_df['CustomerId'].dtype

string[python]

## **Extract data from account info sheet in messy excel**

In [15]:
account_df = pd.read_excel(file_path,sheet_name=1)
account_df = pd.DataFrame(account_df)

In [16]:
account_df.head()

,CustomerId,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,€0.0,1,Yes,2,Yes,1
1,15634602,€0.0,1,Yes,2,Yes,1
2,15647311,€83807.86,1,Yes,1,Yes,0
3,15619304,€159660.8,3,No,8,No,1
4,15701354,€0.0,2,No,1,No,0


In [17]:
account_df['CustomerId'].dtype

dtype('int64')

In [19]:
account_df['CustomerId'] = account_df['CustomerId'].astype('string')

In [20]:
account_df['CustomerId'].dtype

string[python]

## **Merge Data from account and customer**

In [21]:
bank_account_df = account_df.merge(customer_df,on='CustomerId',how='left')

In [27]:
account_df['CustomerId'].count()

np.int64(10002)

In [28]:
customer_df['CustomerId'].count()

np.int64(10001)

In [26]:
bank_account_df['CustomerId'].count()

np.int64(10004)

## **Check Duplicate Rows**

In [30]:
bank_account_df[bank_account_df.duplicated(keep=False)]

,CustomerId,Balance,NumOfProducts,HasCrCard,Tenure_x,IsActiveMember,Exited,Surname,CreditScore,Geography,Gender,Age,Tenure_y,EstimatedSalary
0,15634602,€0.0,1,Yes,2,Yes,1,Hargrave,619,FRA,Female,42.0,2,€101348.88
1,15634602,€0.0,1,Yes,2,Yes,1,Hargrave,619,FRA,Female,42.0,2,€101348.88
10000,15628319,€130142.79,1,No,4,No,0,Walker,792,French,Female,28.0,4,€38190.78
10001,15628319,€130142.79,1,No,4,No,0,Walker,792,French,Female,28.0,4,€38190.78
10002,15628319,€130142.79,1,No,4,No,0,Walker,792,French,Female,28.0,4,€38190.78
10003,15628319,€130142.79,1,No,4,No,0,Walker,792,French,Female,28.0,4,€38190.78


In [31]:
bank_account_df = bank_account_df.drop_duplicates()

In [32]:
bank_account_df[bank_account_df.duplicated(keep=False)]

,CustomerId,Balance,NumOfProducts,HasCrCard,Tenure_x,IsActiveMember,Exited,Surname,CreditScore,Geography,Gender,Age,Tenure_y,EstimatedSalary


In [33]:
bank_account_df['CustomerId'].count()

np.int64(10000)

In [34]:
bank_account_df.columns = bank_account_df.columns.str.strip().str.lower().str.replace(' ','_')

In [35]:
bank_account_df.columns

Index(['customerid', 'balance', 'numofproducts', 'hascrcard', 'tenure_x',
       'isactivemember', 'exited', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure_y', 'estimatedsalary'],
      dtype='object')

## **Check Duplicate Columns**

In [37]:
(bank_account_df['tenure_x'] == bank_account_df['tenure_y']).unique()

array([ True])

In [38]:
bank_account_df.drop(columns='tenure_y',inplace=True)

In [39]:
bank_account_df.columns

Index(['customerid', 'balance', 'numofproducts', 'hascrcard', 'tenure_x',
       'isactivemember', 'exited', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'estimatedsalary'],
      dtype='object')

## **Standardizing Column Name for preventing coding error**

In [43]:
bank_account_df = bank_account_df.rename(columns={'tenure_x':'tenure'})

In [44]:
bank_account_df.columns

Index(['customerid', 'balance', 'numofproducts', 'hascrcard', 'tenure',
       'isactivemember', 'exited', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'estimatedsalary'],
      dtype='object')

## **Check DataType Mismatch & Fix**

In [45]:
bank_account_df.dtypes

customerid         string[python]
balance                    object
numofproducts               int64
hascrcard                  object
tenure                      int64
isactivemember             object
exited                      int64
surname                    object
creditscore                 int64
geography                  object
gender                     object
age                       float64
estimatedsalary            object
dtype: object

In [46]:
bank_account_df['hascrcard'].unique()

array(['Yes', 'No'], dtype=object)

In [51]:
bank_account_df['estimatedsalary']

0        €101348.88
2        €112542.58
3        €113931.57
4         €93826.63
5          €79084.1
            ...    
9996      €96270.64
9997     €101699.77
9998      €42085.58
9999      €92888.52
10000     €38190.78
Name: estimatedsalary, Length: 10000, dtype: string

In [50]:
bank_account_df[['estimatedsalary','balance']] = bank_account_df[['estimatedsalary','balance']].astype('string')

In [52]:
bank_account_df['estimatedsalary'].astype(float)

ValueError: could not convert string to float: '€101348.88'

In [54]:
bank_account_df[['estimatedsalary','balance']] = bank_account_df[['estimatedsalary','balance']].replace(r'[€,]','',regex=True)

In [56]:
bank_account_df[['estimatedsalary','balance']] = bank_account_df[['estimatedsalary','balance']].astype(float)

In [57]:
bank_account_df.dtypes

customerid         string[python]
balance                   float64
numofproducts               int64
hascrcard                  object
tenure                      int64
isactivemember             object
exited                      int64
surname                    object
creditscore                 int64
geography                  object
gender                     object
age                       float64
estimatedsalary           float64
dtype: object

In [58]:
bank_account_df['hascrcard'].unique()

array(['Yes', 'No'], dtype=object)

In [59]:
bank_account_df['isactivemember'].unique()

array(['Yes', 'No'], dtype=object)

In [61]:
bank_account_df[['hascrcard','isactivemember']] = bank_account_df[['hascrcard','isactivemember']].astype('string').replace({'Yes':1,'No':0})

In [62]:
bank_account_df[['hascrcard','isactivemember']] = bank_account_df[['hascrcard','isactivemember']].astype('int')

In [65]:
bank_account_df[['geography','gender']] = bank_account_df[['geography','gender']].astype('string')

In [66]:
bank_account_df['geography'] = bank_account_df['geography'].str.lower().str.strip()

In [67]:
bank_account_df['gender'] = bank_account_df['gender'].str.lower().str.strip()

In [70]:
bank_account_df.map(type).nunique()

customerid         1
balance            1
numofproducts      1
hascrcard          1
tenure             1
isactivemember     1
exited             1
surname            2
creditscore        1
geography          1
gender             1
age                1
estimatedsalary    1
dtype: int64

In [71]:
bank_account_df.isnull().sum()

customerid         0
balance            0
numofproducts      0
hascrcard          0
tenure             0
isactivemember     0
exited             0
surname            3
creditscore        0
geography          0
gender             0
age                3
estimatedsalary    0
dtype: int64

In [72]:
bank_account_df[bank_account_df['age'].isnull()]

,customerid,balance,numofproducts,hascrcard,tenure,isactivemember,exited,surname,creditscore,geography,gender,age,estimatedsalary
29,15728693,141349.43,1,1,3,1,0,NaN,574,germany,female,NaN,-999999.0
122,15580203,120193.42,1,0,6,0,0,NaN,674,spain,male,NaN,-999999.0
9390,15756954,0.00,1,1,2,1,0,NaN,538,france,female,NaN,-999999.0


In [83]:
bank_account_df = bank_account_df.dropna(subset=['balance','age','estimatedsalary','creditscore'])

In [86]:
bank_account_df['age'] = bank_account_df['age'].astype(int)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11052\4088335478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_account_df['age'] = bank_account_df['age'].astype(int)


In [87]:
bank_account_df.dtypes

customerid         string[python]
balance                   float64
numofproducts               int64
hascrcard                   int64
tenure                      int64
isactivemember              int64
exited                      int64
surname                    object
creditscore                 int64
geography          string[python]
gender             string[python]
age                         int64
estimatedsalary           float64
dtype: object

In [90]:
bank_account_df = bank_account_df.drop(columns=['customerid','surname'])

In [92]:
bank_account_df.map(type).nunique()

balance            1
numofproducts      1
hascrcard          1
tenure             1
isactivemember     1
exited             1
creditscore        1
geography          1
gender             1
age                1
estimatedsalary    1
dtype: int64

## **Standardize categorical data**

In [93]:
bank_account_df['geography'].unique()

<StringArray>
['fra', 'spain', 'french', 'france', 'germany']
Length: 5, dtype: string

In [96]:
bank_account_df['geography'] = bank_account_df['geography'].replace(['fra','french'],'france')

In [97]:
bank_account_df['geography'].unique()

<StringArray>
['france', 'spain', 'germany']
Length: 3, dtype: string

In [98]:
bank_account_df['gender'].unique()

<StringArray>
['female', 'male']
Length: 2, dtype: string

In [99]:
bank_account_df.describe()

,balance,numofproducts,hascrcard,tenure,isactivemember,exited,creditscore,age,estimatedsalary
count,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000
mean,76482.679807,1.530359,0.515055,5.013204,0.515055,0.203761,650.545364,38.922077,100092.222656
std,62397.174721,0.581669,0.499798,2.892364,0.499798,0.402814,96.657932,10.489072,57518.775702
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,350.000000,18.000000,11.580000
25%,0.000000,1.000000,0.000000,3.000000,0.000000,0.000000,584.000000,32.000000,50974.570000
50%,97188.620000,1.000000,1.000000,5.000000,1.000000,0.000000,652.000000,37.000000,100236.020000
75%,127642.440000,2.000000,1.000000,7.000000,1.000000,0.000000,718.000000,44.000000,149399.700000
max,250898.090000,4.000000,1.000000,10.000000,1.000000,1.000000,850.000000,92.000000,199992.480000
